In [ ]:
#logistic regression

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score



# Load the data
data = pd.read_csv('pseudodata_præoperation.csv')

#make pd to np
X = data.to_numpy()
y = np.random.choice([0, 1], size=len(data))

In [ ]:
from sklearn.model_selection import KFold, LeaveOneOut
model = LogisticRegression()
N, M = X.shape
k = 10
K = 10
Inner_train_error = np.zeros((k))
Inner_validation_error = np.zeros((k))
coefficient_norm = np.zeros((k))
Inner_gen_errors = np.zeros((k))
Inner_gen_errors_sum = np.zeros((K))
Mean_inner_train_errors = np.zeros((K))
Outer_train_error = np.zeros((K))
Outer_test_error = np.zeros((K))
Outer_gen_errors = np.zeros((K))
Gen_error = 0
# Define the number of folds

# Create the outer loop for the first level of cross-validation
outer_cv = KFold(n_splits=K, shuffle=True)
inner_cv = KFold(n_splits=K, shuffle=True)

outer_results = []


for Outer_count, (train_index, test_index) in enumerate(outer_cv.split(X,y)):
    print('\nCrossvalidation fold: {0}/{1}'.format(Outer_count+1,K))
    X_train = X[train_index,:]
    y_train = y[train_index]
    X_test = X[test_index,:]
    y_test = y[test_index]
    
    # Iterate over the inner folds
    for Inner_count, (train_index_1, test_index_1) in enumerate(inner_cv.split(X_train, y_train)):
        X_train_1 = X_train[train_index_1,:]
        y_train_1 = y_train[train_index_1]
        X_test_1 = X_train[test_index_1,:]
        y_test_1 = y_train[test_index_1]
        
        mdl = LogisticRegression(penalty='l2')

        mdl.fit(X_train_1, y_train_1)

        y_train_est = mdl.predict(X_train_1).T
        y_test_est = mdl.predict(X_test_1).T
        
        # Compute errors:
        Inner_Train_error = np.sum(y_train_est != y_train_1) / len(y_train_1)
        Inner_Validation_error = np.sum(y_test_est != y_test_1) / len(y_test_1)
        
        # Save errors:
        Inner_train_error[Inner_count] = Inner_Train_error
        Inner_validation_error[Inner_count] = Inner_Validation_error
        Inner_Gen_error = (len(y_test_1)/len(y_test))*Inner_Validation_error
        Inner_gen_errors[Inner_count,] = Inner_Gen_error 
        Inner_gen_errors_sum[Inner_count] = Inner_gen_errors_sum[Inner_count] + Inner_Gen_error

        # Computing the Gen_s error:
    Mean_inner_train_errors[Outer_count] = np.mean(Inner_train_error)
    Inner_gen_errors_sum[Outer_count] = np.sum(Inner_gen_errors)
    
    # Fit logistic regression model
    mdl = LogisticRegression(penalty='l2')

    mdl.fit(X_train, y_train)

    y_train_est = mdl.predict(X_train).T
    y_test_est = mdl.predict(X_test).T

    Outer_Train_error = np.sum(y_train_est != y_train) / len(y_train)
    Outer_Validation_error = np.sum(y_test_est != y_test) / len(y_test)

    Outer_train_error[Outer_count] = Outer_Train_error
    Outer_test_error[Outer_count] = Outer_Validation_error
    Outer_Gen_error = (len(y_test)/N)*Outer_Validation_error
    Outer_gen_errors[Outer_count] = Outer_Gen_error
    Gen_error += Outer_Gen_error
print(f"\nGen error: {Gen_error}")
    


    
